In [1]:
#pip install googletrans

# at the date of 03/02/21, there is an issue with official pip librarie ''NoneType' object has no attribute 'group''
# temporary solution :
#pip install googletrans==3.1.0a0

In [3]:
import pandas as pd

In [4]:
# LOAD POPULATION FILE
dfpop = pd.read_csv("population_2019.csv",index_col=0)
dfpop.head()

,population
country,
Afghanistan,38
Albania,2.9
Algeria,43.1
American Samoa,0.1
Andorra,0.1


In [5]:
# LOAD API COUNTRY NAME
dfapi = pd.read_csv("countriesNameAPI.csv",index_col=1)
print(dfapi.columns)
dfapi.drop(columns=[dfapi.columns[0]],inplace=True)
dfapi.head()

Index(['Unnamed: 0', 'Slug', 'ISO2'], dtype='object')


,Slug,ISO2
Country,,
Peru,peru,PE
Andorra,andorra,AD
Bahamas,bahamas,BS
Gabon,gabon,GA
Kazakhstan,kazakhstan,KZ


In [6]:
# Join tables with inner to have right callname for api AND population number
joincountry= pd.concat([dfpop,dfapi], axis=1,join='inner').drop(columns=['Slug', 'ISO2'])
joincountry.head(20)

,population
Afghanistan,38
Albania,2.9
Algeria,43.1
American Samoa,0.1
Andorra,0.1
Angola,31.8
Antigua and Barbuda,0.1
Argentina,44.9
Armenia,3
Aruba,0.1


In [7]:
# ADD column with translation of country name in french
from googletrans import Translator
translator = Translator()
# get index list
countrylist = joincountry.index
countrylist

Index(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       ...
       'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom',
       'United States of America', 'Uruguay', 'Uzbekistan', 'Vanuatu',
       'Zambia', 'Zimbabwe'],
      dtype='object', length=182)

In [10]:
#TEST UNITAIRE
result = translator.translate('whats up', src='en',dest='fr')
print(result.text)

quoi de neuf


In [13]:
# GETTING TRANSLATION FOR ALL COUNTRY LIST
translations = translator.translate(list(countrylist), src='en', dest='fr')

In [15]:
# FILLING COUNTRYLISTFR
countrylistfr = []
for elem in translations:
    print(elem.text)
    countrylistfr.append(elem.text)

Afghanistan
Albanie
Algérie
Samoa américaines
Andorre
Angola
Antigua-et-Barbuda
Argentine
Arménie
Aruba
Australie
L'Autriche
Azerbaïdjan
Bahreïn
Bangladesh
Barbade
Biélorussie
Belgique
Belize
Bénin
Bermudes
Bhoutan
Bolivie
Bosnie Herzégovine
Botswana
Brésil
Brunei Darussalam
Bulgarie
Burkina Faso
Burundi
Cambodge
Cameroun
Canada
Îles Caïmans
République centrafricaine
Tchad
Chili
Chine
Colombie
Comores
Costa Rica
Croatie
Cuba
Chypre
République Tchèque
Danemark
Djibouti
Dominique
République dominicaine
Equateur
Le Salvador
Guinée Équatoriale
Érythrée
Estonie
Ethiopie
Îles Féroé
Fidji
Finlande
France
Polynésie française
Gabon
Géorgie
Allemagne
Ghana
Grèce
Groenland
Grenade
Guam
Guatemala
Guinée
Guinée-Bissau
Guyane
Haïti
Honduras
Hongrie
Islande
Inde
Indonésie
Irak
Irlande
île de Man
Israël
Italie
Jamaïque
Japon
Jordan
Kazakhstan
Kenya
Kiribati
Koweit
République démocratique populaire lao
Lettonie
Liban
Lesotho
Libéria
Libye
Liechtenstein
Lituanie
Luxembourg
Madagascar
Malawi
Malaisie
Mal

In [17]:
# adding column to dataframe
joincountry['countryfr'] = countrylistfr

In [18]:
joincountry.head()

,population,countryfr
Afghanistan,38,Afghanistan
Albania,2.9,Albanie
Algeria,43.1,Algérie
American Samoa,0.1,Samoa américaines
Andorra,0.1,Andorre


In [19]:
joincountry.to_csv('country_concat_inner_translate.csv') 

In [34]:
# TEST FOR MODELING
pdc = pd.read_csv("country_concat_inner_translate_continent.csv")
pdc.head()

,country,population,countryfr,continent
0,Afghanistan,38.0,Afghanistan,Asie
1,South Africa,58.6,Afrique du Sud,Afrique
2,Albania,2.9,Albanie,Europe
3,Algeria,43.1,Algérie,Afrique
4,Germany,83.1,Allemagne,Europe


In [49]:
Continentlist = pdc['continent'].unique()
print(Continentlist)

['Asie' 'Afrique' 'Europe' 'Amérique' 'Océanie']


In [50]:
Continentlist.sort()
print(Continentlist)

['Afrique' 'Amérique' 'Asie' 'Europe' 'Océanie']


In [51]:
def get_countrymap(pdc):
    pdc_filt = pdc.sort_values('countryfr')
    resultdict = {}
    for row in pdc_filt.iterrows():
        resultdict[row[1]['country']] = row[1]['countryfr']
    return resultdict

test = get_countrymap(pdc)
#print(test)

In [61]:
def get_continentmapandlist(pdc):
    Continentlist = pdc['continent'].unique()
    Continentlist.sort()
    
    resultdict = {}
    for continentname in Continentlist:
        pdextract = pdc[['country','countryfr']].loc[(pdc['continent'] == continentname)]
        #print(pdextract.head())
        somedict = {row[1]['country']:row[1]['countryfr'] for row in pdextract.iterrows()}
        resultdict[continentname] = somedict
    return resultdict,list(Continentlist)

testmap,testlist = get_continentmapandlist(pdc)


In [62]:
testmap

{'Afrique': {'South Africa': 'Afrique du Sud',
  'Algeria': 'Algérie',
  'Angola': 'Angola',
  'Benin': 'Bénin',
  'Botswana': 'Botswana',
  'Burkina Faso': 'Burkina Faso',
  'Burundi': 'Burundi',
  'Cameroon': 'Cameroun',
  'Comoros': 'Comores',
  'Djibouti': 'Djibouti',
  'Ethiopia': 'Ethiopie',
  'Gabon': 'Gabon',
  'Ghana': 'Ghana',
  'Guinea': 'Guinée',
  'Equatorial Guinea': 'Guinée Équatoriale',
  'Guinea-Bissau': 'Guinée-Bissau',
  'Mauritius': 'Ile Maurice',
  'Kenya': 'Kenya',
  'Seychelles': 'les Seychelles',
  'Lesotho': 'Lesotho',
  'Liberia': 'Libéria',
  'Libya': 'Libye',
  'Madagascar': 'Madagascar',
  'Malawi': 'Malawi',
  'Mali': 'Mali',
  'Morocco': 'Maroc',
  'Mauritania': 'Mauritanie',
  'Mozambique': 'Mozambique',
  'Namibia': 'Namibie',
  'Niger': 'Niger',
  'Nigeria': 'Nigeria',
  'Uganda': 'Ouganda',
  'Central African Republic': 'République centrafricaine',
  'Rwanda': 'Rwanda',
  'Sao Tome and Principe': 'Sao Tomé et Principe',
  'Senegal': 'Sénégal',
  'Sier

In [63]:
testlist

['Afrique', 'Amérique', 'Asie', 'Europe', 'Océanie']